# 학습 목표
1. Alphafold의 주요 원리와 구조를 이해한다.
2. ColabFold(AlphaFold2 오픈소스 버전)를 활용해 실제 단백질 구조 예측을 실행해본다.
3. 예측된 3차원 단백질 구조를 시각화하고, 신뢰도(pLDDT) 등을 해석하는 방법을 익힌다.

<img src='https://www.researchgate.net/publication/365398397/figure/fig3/AS:11431281097310182@1668533124442/The-AlphaFold-and-AlphaFold2-results-at-CASP13-and-CASP14-respectively.ppm'>

# 1.  MSA (Multiple Sequence Alignment)
**다중 서열 정렬(MSA, Multiple Sequence Alignment)**은 서로 상동(homologous) 관계에 있는 단백질(또는 DNA/RNA) 서열들을 세로로 정렬하여,
- 어느 위치에서 아미노산이 어떻게 변했는지,
- 어떤 자리에서 변이가 적고 보존(Conserved)되어 있는지,
- 두 잔기가 함께 변하는 패턴(공진화 Co-evolution)이 존재하는지

등을 한눈에 파악하도록 도와주는 기법

<img src='https://bioinformaticsreview.com/wp-content/uploads/2015/10/msa-example.png'>

# 2. AlphaFold(2018)
- AlphaFold1은 CASP13(2018)에서 1위를 차지함.
- 핵심 아이디어:
    - MSA에서 잔기 쌍의 거리를 확률 분포로 예측 (예: “Residue i와 j는 6Å, 7Å, 8Å일 확률은 각각 얼마”)
    - CNN(ResNet)으로 공진화 신호를 학습하여 “거리분포를 출력”
    - 예측된 거리분포 전체를 통계 포텐셜로 간주해 에너지 최소화(경사하강법) 수행 → 3차원 구조를 산출

<img src='https://github.com/Charmve/AlphaFold-baseline/raw/main/imgs/fold-process.png'>

# 3. AlphaFold2(2020)
- AlphaFold2는 CASP14(2020)에서 이전 세대를 뛰어넘는 정확도로 우승
- 약 100개의 평가 단백질 중 대부분을 높은 정밀도로 예측했고, RMSD가 1Å대에 이르는 경우도 많아, 실험 구조와 구별하기 어려운 수준에 달함.
- AlphaFold1처럼 “거리분포 → 별도 에너지 최소화”라는 분리된 단계를 두지 않고,
하나의 신경망 안에서 거리/각도/3D 좌표를 연속적으로 학습하여 바로 구조를 산출
- 핵심 아이디어:
    - Evoformer: Transformer 기반 블록을 다중 반복하여 MSA 표현
    - Structure Module: end-to-end로 3차원 좌표를 직접 예측
    - Recycling: 예측된 구조를 다시 입력으로 넣어 (기본 3회 반복) 점차 구조 정확도를 개선

<img src='https://i0.wp.com/www.blopig.com/blog/wp-content/uploads/2021/07/image-3.png?ssl=1'>

# 4. ColabFold를 활용한 AlphaFold 예측
ColabFold는 다음 단계로 동작:
- MSA 검색: 해당 단백질 서열과 유사한 서열들을 원격 DB(MMseqs2 등)에서 검색 → MSA 구성
- Template 검색(옵션): PDB에서 유사 구조 템플릿 검색
- AlphaFold 모델 예측: 사전학습된 AlphaFold2 파라미터 사용
- 결과 저장: PDB 파일, 예측 점수(ranking_debug.json), MSA 폴더 등
- 완료되면, 다음 파일들이 생성:
    - _unrelaxed_model_1.pdb ~ _model_5.pdb: 모델별 예측 구조
    - _scores.json: 각 모델의 예측 점수(pLDDT 등)
    - ranking_debug.json: 5개 모델 중 어떤 것이 높은 점수인지 기록
    - msas/: 검색된 MSA 관련 파일들

https://github.com/sokrypton/ColabFold?tab=readme-ov-file

https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/batch/AlphaFold2_batch.ipynb

https://colab.research.google.com/github/sokrypton/ColabDesign/blob/gamma/af/examples/predict.ipynb

# 5. 결과 해석
- pLDDT (predicted LDDT) 점수
    - 0~100 범위
    - 90 이상이면 실험 구조에 매우 근접할 가능성이 큼
    - 루프나 말단 부위 등은 상대적으로 점수가 낮을 수 있음

- 서열 길이
    - 매우 긴 단백질(수천 잔기)은 Colab에서 예측이 오래 걸림
    - 긴 도메인은 여러 조각으로 나눠서 예측하거나, Colab Pro+ 같은 고성능 환경을 이용
- relaxation (물리적 후처리)
    - AlphaFold 예측 좌표에 대해 OpenMM으로 살짝 에너지 최소화를 적용할 수 있음 (amber_relax)
    - 단백질 고유의 미세 구조 조정에 도움을 주지만, 전체 구조 변동은 제한적